In [2]:
import sqlite3
import pandas as pd
import logging
import sys

# Logging
logging.basicConfig(
    filename=r"C:\Users\lenovo\OneDrive\Desktop\vendor\logs\get_vendor_summary.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

# Connect DB
conn = sqlite3.connect(r"C:\Users\lenovo\OneDrive\Desktop\vendor\inventory.db")

# ---------------------------------------------------------
# ✅ Step 1 — SQL Query with corrected SalesQuantity column
# ---------------------------------------------------------
query = """
WITH 
freight_summary AS (
    SELECT 
        VendorNumber, 
        SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),
purchase_summary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        AVG(p.PurchasePrice) AS AvgPurchasePrice,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars,
        AVG(pp.Price) AS AvgActualPrice,
        SUM(pp.Volume) AS TotalVolume
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY
        p.VendorNumber, p.VendorName, p.Brand, p.Description
),
sales_summary AS (
    SELECT
        VendorNo,
        Brand,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(ExciseTax) AS TotalExciseTax
    FROM Sales
    GROUP BY VendorNo, Brand
),
Vendor_Sales_Summary AS (
    SELECT
        ps.VendorNumber,
        ps.VendorName,
        ps.Brand,
        ps.Description,
        ps.AvgPurchasePrice,
        ps.TotalPurchaseQuantity,
        ps.TotalPurchaseDollars,
        ps.AvgActualPrice,
        ps.TotalVolume,
        ss.TotalSalesDollars,
        ss.TotalSalesPrice,
        ss.TotalSalesQuantity,
        ss.TotalExciseTax,
        fs.FreightCost,
        (IFNULL(ss.TotalSalesDollars, 0) 
         - IFNULL(ps.TotalPurchaseDollars, 0)  
         - IFNULL(fs.FreightCost, 0)) AS EstimatedProfit
    FROM purchase_summary ps
    LEFT JOIN sales_summary ss
        ON ps.VendorNumber = ss.VendorNo 
        AND ps.Brand = ss.Brand
    LEFT JOIN freight_summary fs
        ON ps.VendorNumber = fs.VendorNumber
)
SELECT *
FROM Vendor_Sales_Summary
ORDER BY TotalPurchaseDollars DESC;
"""

logging.info("📥 Running Vendor Sales Summary Query...")
vendor_sales_summary = pd.read_sql_query(query, conn)
logging.info("📥 Query executed successfully!")

# ---------------------------------------------------------
# ✅ Step 2 — Clean & Enhance
# ---------------------------------------------------------
df = vendor_sales_summary.copy()
df.fillna(0, inplace=True)

# New Calculated Columns
df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']

df['ProfitMargin'] = df.apply(
    lambda x: (x['GrossProfit'] / x['TotalSalesDollars'] * 100)
    if x['TotalSalesDollars'] != 0 else 0,
    axis=1
)

df['SalesToPurchaseRatio'] = df.apply(
    lambda x: (x['TotalSalesDollars'] / x['TotalPurchaseDollars'])
    if x['TotalPurchaseDollars'] != 0 else 0,
    axis=1
)

df['StockTurnover'] = df.apply(
    lambda x: (x['TotalSalesQuantity'] / x['TotalPurchaseQuantity'])
    if x['TotalPurchaseQuantity'] != 0 else 0,
    axis=1
)

# ---------------------------------------------------------
# ✅ Step 3 — Save back to DB
# ---------------------------------------------------------
df.to_sql("vendor_sales_summary_clean", conn, if_exists="replace", index=False)
print("✅ Saved cleaned table as vendor_sales_summary_clean")

print(df.head())


✅ Saved cleaned table as vendor_sales_summary_clean
   VendorNumber                   VendorName  Brand              Description  \
0          1128  BROWN-FORMAN CORP             1233  Jack Daniels No 7 Black   
1          4425        MARTIGNETTI COMPANIES   3405    Tito's Handmade Vodka   
2         17035  PERNOD RICARD USA             8068         Absolut 80 Proof   
3          3960  DIAGEO NORTH AMERICA INC      4261   Capt Morgan Spiced Rum   
4          3960  DIAGEO NORTH AMERICA INC      3545          Ketel One Vodka   

   AvgPurchasePrice  TotalPurchaseQuantity  TotalPurchaseDollars  \
0             26.27                 768207           20180797.89   
1             23.19                 863895           20033725.05   
2             18.24                1005435           18339134.40   
3             16.17                1022724           16537447.08   
4             21.89                 721290           15789038.10   

   AvgActualPrice  TotalVolume  TotalSalesDollars  TotalSa

In [3]:
import sqlite3
import pandas as pd

# connect to your DB
conn = sqlite3.connect(r"C:\Users\lenovo\OneDrive\Desktop\vendor\inventory.db")

# list all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("📂 Tables in inventory.db:")
print(tables)


📂 Tables in inventory.db:
                         name
0              vendor_invoice
1             purchase_prices
2                       sales
3             begin_inventory
4               end_inventory
5                   purchases
6        vendor_sales_summary
7  vendor_sales_summary_clean


In [4]:
pd.read_sql_query("PRAGMA table_info(vendor_sales_summary_clean);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,VendorNumber,INTEGER,0,None,0
1,1,VendorName,TEXT,0,None,0
2,2,Brand,INTEGER,0,None,0
3,3,Description,TEXT,0,None,0
4,4,AvgPurchasePrice,REAL,0,None,0
5,5,TotalPurchaseQuantity,INTEGER,0,None,0
6,6,TotalPurchaseDollars,REAL,0,None,0
7,7,AvgActualPrice,REAL,0,None,0
8,8,TotalVolume,REAL,0,None,0
9,9,TotalSalesDollars,REAL,0,None,0


In [5]:
#code to download the cleaned vendor summary table as a CSV file
df = pd.read_sql_query("SELECT * FROM vendor_sales_summary_clean;", conn)   